<a href="https://colab.research.google.com/github/The-Geology-Guy/sample_determination/blob/main/SampleDetermination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Null Hypothesis Significance Testing (NHST)

##### _March, 2019_

PAJER, Luke. luke.pajer@gmail.com

---------------------



In [1]:
import numpy as np
import pandas as pd
import scipy.stats
from IPython.display import display, Markdown, Latex
pd.set_option('display.max_colwidth', 0)

#### Determine Alpha Function

In [2]:
def alpha_cv(a):
    alpha_dict = {"0.10": 1.282, "0.05": 1.645, "0.025": 1.960, "0.01": 2.326, "0.005": 2.576, "0.001": 3.090, "0.0005": 3.291}
    return alpha_dict.get(a, "")

#### Determine Rejection of Null Hypothesis Function

In [3]:
def rejection(z_cv, z_alpha, rr):
    if (rr == 'upper'):
        test = (z_cv >= z_alpha)
    elif (rr == 'lower'):
        test = (z_cv <= (-1 * z_alpha))
    elif (rr == 'two'):
        test = ((z_cv >= z_alpha) & (z_cv <= z_alpha))
    return test

#### Explain the Rejection Function

In [4]:
def rej_explain(z, a, alpha, r, rr):
    if ((r == True) & (rr == 'upper')):
        explain = (str(np.round(z, 3)) + " $\geq$ " + str(a) + 
                   r": $\text{Reject}$ $H_{0}$ because the Test statistic $(z)$ falls in the Rejection Region ($z$ $\geq$ $z_{\alpha}$ = " + 
                   str(alpha) + ").")
    elif ((r == True) & (rr == 'lower')):
        explain = (str(np.round(z, 3)) + " $\leq$ -" + str(a) + 
                   r": $\text{Reject}$ $H_{0}$ because the Test statistic $(z)$ falls in the Rejection Region ($z$ $\leq$ $-z_{\alpha}$ = " + 
                   str(alpha) + ").")
    elif ((r == True) & (rr == 'two')):
        explain = ""
    elif ((r == False) & (rr == 'upper')):
        explain = (str(np.round(z, 3)) + " $\leq$ " + str(a) + 
                   r": $\text{Do not reject}$ $H_{0}$ because the Test statistic $(z)$ does not fall in the Rejection Region ($z$ $\geq$ $z_{\alpha}$ = " + 
                   str(alpha) + ").")
    elif ((r == False) & (rr == 'lower')):
        explain = (str(np.round(z, 3)) + " $\geq$ -" + str(a) + 
                   r": $\text{Do not reject}$ $H_{0}$ because the Test statistic $(z)$ does not fall in the Rejection Region ($z$ $\leq$ $-z_{\alpha}$ = " + 
                   str(alpha) + ").")
    elif ((r == False) & (rr == 'two')):
        explain = ""
    return display(Markdown(explain))

#### Compute the Test Statistic Value

In [5]:
def ztest(p_hat, p_0, nt, alpha, reject_region):
    valid_head = ((nt * p_0) >= 10)
    valid_tail = ((nt * (1 - p_0)) >= 10)
    if (valid_head & valid_tail):
        zt = (p_hat - p_0)/(np.sqrt((p_0 *(1 - p_0))/nt))
        alpha_1 = alpha_cv(alpha)
        answer = rejection(zt, alpha_1, reject_region)
        return rej_explain(zt, alpha_1, alpha, answer, reject_region)
    else:
        return display(Markdown("Does not meet the test procedures requirements"))

#### Solve for Sample Size needed

_Take a look into the beta error function, does it need to be subtracted by one?_

In [6]:
def sampleDeter(p0, p1, za, ns):
    za = alpha_cv(za)
    phi_z = (p0 - p1 + za * np.sqrt(abs(p0 * (1 - p0))/ns)) / np.sqrt(abs((p1 * (1 - p1)))/ns)
    beta_err = scipy.stats.norm.cdf(phi_z)
    n = (((za * np.sqrt(p0 * (1 - p0))) + (za * np.sqrt(p1 * (1 - p1)))) / (p1 - p0)) ** 2
    retest = p0 + (za * np.sqrt((p0 * (1-p0))/ns))
    return pd.DataFrame.from_dict({r'$\beta$': [np.round(beta_err, 3)], 
                                   r"$n$": [int(np.round(n, 0))], 
                                   r"$z$": [np.round(phi_z, 3)], 
                                   r"$p$": [np.round(p0, 3)],
                                   r"$p'$": [np.round(p1, 3)],
                                   r"$c$": [np.round(retest, 3)]
                                  })

### Run the NHST for each Type I Error $(\alpha)$

In [7]:
def to_md(p0, p1, n, tail):
    alpha_list = ["0.10", "0.05", "0.025", "0.01", "0.005", "0.001", "0.0005"]
    i = 0
    while i < len(alpha_list):
        alpha_a = alpha_list[i]
        display(Markdown("--------------------"))
        display(Markdown(r"$\text{For Type I Error } (\alpha)$ = " + alpha_a))
        ztest(p1, p0, n, str(alpha_a), tail)
        test = sampleDeter(p0, p1, str(alpha_a), n)
        beta_, n_, z_, p0_, p1_, c_ = test.iloc[0,0], test.iloc[0,1], test.iloc[0,2], test.iloc[0,3], test.iloc[0,4], test.iloc[0,5]
        display(Markdown(r"If the experiment collects data on $n =$ " + str(n_) + r" users, and $\text{Rejects}$ $H_{0}: p = $ " 
                                + str(float(round(p0, 3))) + " if the proportion of sampled users who favor the change is greater than " + str(c_) + 
                                ", then there will be a " + str(int(round(float(alpha_a), 2) * 100)) + 
                                r"% chance of committing a Type I error $(\alpha)$ and a " + 
                                str(int(round(beta_, 2) * 100)) + r"% chance of committing a Type II error $(\beta)$ if the population proportion were actually " + 
                                str(p1_) + "."))
        display(Markdown(r"$\hat{p} =$ " + str(float(round(p0, 3))) + r", $p =$ " + str(float(round(p1, 3))) + r", $c =$ " + str(c_)))
        display(Markdown(r"$n =$ " + str(n) + r", $n_{c} =$ " + str(n_)))
        display(Markdown(r"$\alpha =$ " + str(alpha_a) + r", $\beta =$ " + str(beta_)))
        i +=1

#### Enter values for the variables

`p0` is the original or unmodified data to test  
`p1` is the modified data to test  
`n` is the total number of trials for the p1  
`upper` is for predicting an increase  

In [8]:
p0, p1, n, tail, = 0.0301, 0.0487, 256836, "upper"
display(Markdown("### Transaction Analysis"))
to_md(p0, p1, n, tail)

### Transaction Analysis

--------------------

$\text{For Type I Error } (\alpha)$ = 0.10

55.169 $\geq$ 1.282: $\text{Reject}$ $H_{0}$ because the Test statistic $(z)$ falls in the Rejection Region ($z$ $\geq$ $z_{\alpha}$ = 0.10).

If the experiment collects data on $n =$ 708 users, and $\text{Rejects}$ $H_{0}: p = $ 0.03 if the proportion of sampled users who favor the change is greater than 0.031, then there will be a 10% chance of committing a Type I error $(\alpha)$ and a 0% chance of committing a Type II error $(\beta)$ if the population proportion were actually 0.049.

...

55.169 $\geq$ 3.291: $\text{Reject}$ $H_{0}$ because the Test statistic $(z)$ falls in the Rejection Region ($z$ $\geq$ $z_{\alpha}$ = 0.0005).

If the experiment collects data on $n =$ 4667 users, and $\text{Rejects}$ $H_{0}: p = $ 0.03 if the proportion of sampled users who favor the change is greater than 0.031, then there will be a 0% chance of committing a Type I error $(\alpha)$ and a 0% chance of committing a Type II error $(\beta)$ if the population proportion were actually 0.049.

$\hat{p} =$ 0.03, $p =$ 0.049, $c =$ 0.031

$n =$ 256836, $n_{c} =$ 4667

$\alpha =$ 0.0005, $\beta =$ 0.0